In [4]:
import requests
import json
import pickle

In [2]:
#First request, API key removed for safety purposes
request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY")
request.status_code

200

In [31]:
#Converting the variable into a dictionary
data_dict=json.loads(request.text)

In [ ]:
#Inspecting the data
data_dict['results']
print(len(data_dict['results'])) #Only 20 restaurants => need to send additional requests for subsequent pages

In [43]:
#Preparing variables for additional requests
next_page_token=json.loads(request.text)['next_page_token']
restaurants=[]
restaurants.extend(json.loads(request.text)['results'])

In [ ]:
#Sending additional requests
i=0
while next_page_token:
    new_request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY&pageToken={next_page_token}")
    if new_request.status_code==200:
        restaurants.extend(json.loads(new_request.text)['results'])
        next_page_token=json.loads(new_request.text)['next_page_token']
        i += 1
    else:
        print(i)
        print(new_request.status_code)
        break
#Note: the loop was broken since I have reached the free quota
#Note 2: API key removed for safety purposes

In [75]:
#Filtering the data
#Empty list
restaurants_filtered=[]
#Keys
keys_new_dict=['name','address','opening_hours','place_id','price_level','rating','no_of_reviews','location']
keys_old_dict=['name','formatted_address','opening_hours','place_id','price_level','rating','user_ratings_total','geometry']
#Looping through restaurants to filter the data
for restaurant in restaurants:
    dict={}
    for key_new, key_old in zip(keys_new_dict, keys_old_dict):
        if key_new=='location': #Location is a dict within adict
            try:
                dict[key_new]=restaurant[key_old][key_new]
            except KeyError:
                dict[key_new]=None
        else: #Rest of the keys
            try:
                dict[key_new]=restaurant[key_old]
            except KeyError:
                dict[key_new]=None
    restaurants_filtered.append(dict)

In [78]:
#Saving the list for future use
with open('restaurants_google_maps_places_API.pkl', 'wb') as data_file:
    pickle.dump(restaurants_filtered, data_file)

In [ ]:
#Code to read the file
with open('restaurants_google_maps_places_API.pkl.pkl', 'wb') as data_file:
    pickle.dump(restaurants_filtered, data_file)